### Introduction

이번 kaggle 의 [Home Credit - Credit Risk Model Stability](https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability) 를 참여하게 되면서, 처음으로 BigData-Scale Datasets을 다루게 되었다.</br>

평소와 같이 pandas 를 사용하여 여러 EDA 를 진행하는 과정에서 속도가 너무 느리고,
심지어는 멈추는 현상까지 발생하였다. 대회의 public code 에서 많은 유저들이 `polars` library 를 사용하는 것을 확인했고 이는 Rust 기반 라이브러리이다. 러스트는 **Ownership** 이라는 기능을 사용하여 Garbage Collection 에 의존하지 않고 빠르고 안정적인 메모리 관리를 구현한 것으로 알고있다.</br>

pandas 는 아주 많이 사용되고 있어 포기할 수 없지만, 비슷한 기능이 구현된 `polars`를 익혀두어 적절하게 사용할 수 있으면 좋을 것 같다.



## Polars

#### why polars?

pandas 는 훌륭한 라이브러리지만, 가장 많이 이야기가 나오는 문제는 아래와 같다.
- performance and awkward
- complex API.

Polars는 처음에 Richie Vink가 이러한 문제를 해결하기 위해 개발되었고 아래와 같은 철학이 있다고 한다:

> The goal of Polars is to provide a lightning fast DataFrame library that:
> - Utilizes `all available cores` on your machine.
> - Optimizes queries to `reduce unneeded work/memory allocations`. 
> - **Handles datasets much larger than your available RAM.**
> - **A consistent and predictable API.**
> - Adheres to a `strict schema` (data-types should be known before running the query).</br>

> Polars is written in Rust which gives it C/C++ performance and allows it to fully control performance critical parts in a query engine.

추가적으로 성능 측정에 대한 결과는 [polars github](https://github.com/pola-rs/polars) 에 상세히 나와있다.

### Getting Started

In [1]:
# Import the library
import polars as pl
import pandas as pd

In [2]:
# read parquet file
df = pl.read_parquet('./data/parquet_files/train/train_deposit_1.parquet')
pd_df = pd.read_parquet('./data/parquet_files/train/train_deposit_1.parquet')

In [3]:
# check data type
# pandas -> info()
df.schema

OrderedDict([('case_id', Int64),
             ('amount_416A', Float64),
             ('contractenddate_991D', String),
             ('num_group1', Int64),
             ('openingdate_313D', String)])

In [4]:
# head, tail, describe 는 동일하게 제공된다.
df.head()

case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
i64,f64,str,i64,str
225,0.0,null,0,"""2016-08-16"""
331,260.374,"""2018-03-18""",0,"""2015-03-19"""
358,0.0,null,0,"""2014-09-02"""
390,211748.53,"""2017-07-22""",0,"""2014-07-23"""
390,223.68001,null,2,"""2016-06-08"""


In [13]:
df.describe()

statistic,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
str,f64,f64,str,f64,str
"""count""",145086.0,145086.0,"""65404""",145086.0,"""145086"""
"""null_count""",0.0,0.0,"""79682""",0.0,"""0"""
"""mean""",1.4662e6,8422.304482,null,0.522531,null
"""std""",886528.958911,86232.120476,null,1.620954,null
"""min""",225.0,-40000.0,"""2002-02-27""",0.0,"""2001-11-19"""
"""25%""",660041.0,0.0,null,0.0,null
"""50%""",1.556939e6,223.658,null,0.0,null
"""75%""",2.530539e6,478.34,null,1.0,null
"""max""",2.703453e6,1.2213286e7,"""2020-03-18""",64.0,"""2017-07-31"""


polars 라이브러리의 출력에는 몇 가지 특징이 발견된다:
- `shape` 이 출력에 포함되어 실수로 열이나 행을 drop 하는 것을 방지할 수 있다.
- 각 column 의 이름 아래에 `data type` 을 보여준다.
- Index numbers 가 없다.
- 문자열 값에 `""` 를 포함한다.

아래의 pandas DataFrame 과 비교해보면 쉽게 알 수 있다.</br>
개인적으로는 pandas 의 출력보다 polars 의 출력이 더 마음에 든다.


In [16]:
pd_df.head()

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
0,225,0.00000,None,0,2016-08-16
1,331,260.37400,2018-03-18,0,2015-03-19
2,358,0.00000,None,0,2014-09-02
3,390,211748.53000,2017-07-22,0,2014-07-23
4,390,223.68001,None,2,2016-06-08


### Basic concepts
Polars 는 매우 빠른 성능의 핵심인 `Expressions` 라는 개념을 갖고 있다. 이에 해당하는 4가지 API 는 아래와 같다:
- `select` : 원하는 columns 를 선택한다.
- `filter` : operator 를 사용하여 rows 를 추출한다. 
- `with_columns` : 새로운 column 을 생성한다.
- `group_by` pandas 의 **groupby** 와 비슷하지만 **index** 개념이 없다는 것에서 차이가 있다.

#### Selecting and filtering rows and columns

##### `select`

In [22]:
df.select(pl.col('contractenddate_991D', 'openingdate_313D'))

contractenddate_991D,openingdate_313D
str,str
null,"""2016-08-16"""
"""2018-03-18""","""2015-03-19"""
null,"""2014-09-02"""
"""2017-07-22""","""2014-07-23"""
null,"""2016-06-08"""
"""2017-09-30""","""2015-10-01"""
null,"""2016-07-19"""
"""2017-07-31""","""2015-08-01"""
null,"""2016-02-22"""


In [20]:
%%timeit
df.select(pl.col('contractenddate_991D', 'openingdate_313D'))

13.6 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [21]:
%%timeit
pd_df.loc[:, ['contractenddate_991D', 'openingdate_313D']]

2.15 ms ± 27 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


$\rightarrow$ `select()` 평균시간이 약 160배가 차이난다.

##### `filter`

In [23]:
df.filter(pl.col("amount_416A") > 223.658)

case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
i64,f64,str,i64,str
331,260.374,"""2018-03-18""",0,"""2015-03-19"""
390,211748.53,"""2017-07-22""",0,"""2014-07-23"""
390,223.68001,null,2,"""2016-06-08"""
445,23735.938,null,4,"""2016-07-19"""
453,90463.195,null,2,"""2016-03-03"""
582,827.77405,"""2018-10-02""",0,"""2014-04-02"""
713,997.86,"""2018-04-01""",0,"""2015-04-02"""
714,580.248,"""2018-05-25""",0,"""2013-11-26"""
731,232.658,null,0,"""2016-04-21"""


In [24]:
%%timeit
df.filter(pl.col("amount_416A") > 223.658)

495 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
%%timeit
pd_df.loc[pd_df['amount_416A'] > 223.658]

2.51 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Adding columns: `with_columns`

In [26]:
df.head()

case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
i64,f64,str,i64,str
225,0.0,null,0,"""2016-08-16"""
331,260.374,"""2018-03-18""",0,"""2015-03-19"""
358,0.0,null,0,"""2014-09-02"""
390,211748.53,"""2017-07-22""",0,"""2014-07-23"""
390,223.68001,null,2,"""2016-06-08"""


In [29]:
%%time
df = df.with_columns((pl.col('num_group1') == 0).alias('zero_group_1'))
df = df.with_columns((pl.col('contractenddate_991D').str.to_datetime()))
df = df.with_columns((pl.col('openingdate_313D').str.to_datetime()))
df

CPU times: total: 62.5 ms
Wall time: 56 ms


case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D,zero_group_1
i64,f64,datetime[μs],i64,datetime[μs],bool
225,0.0,null,0,2016-08-16 00:00:00,true
331,260.374,2018-03-18 00:00:00,0,2015-03-19 00:00:00,true
358,0.0,null,0,2014-09-02 00:00:00,true
390,211748.53,2017-07-22 00:00:00,0,2014-07-23 00:00:00,true
390,223.68001,null,2,2016-06-08 00:00:00,false
390,203.602,2017-09-30 00:00:00,1,2015-10-01 00:00:00,false
445,23735.938,null,4,2016-07-19 00:00:00,false
445,0.0,2017-07-31 00:00:00,1,2015-08-01 00:00:00,false
445,0.0,null,3,2016-02-22 00:00:00,false


In [30]:
%%time
pd_df['zero_group_1'] = pd_df['num_group1'].apply(lambda x:x==0)
pd_df['contractenddate_991D'] = pd.to_datetime(pd_df['contractenddate_991D'])
pd_df['openingdate_313D'] = pd.to_datetime(pd_df['openingdate_313D'])
pd_df

CPU times: total: 125 ms
Wall time: 107 ms


,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D,zero_group_1
0,225,0.00000,NaT,0,2016-08-16,True
1,331,260.37400,2018-03-18,0,2015-03-19,True
2,358,0.00000,NaT,0,2014-09-02,True
3,390,211748.53000,2017-07-22,0,2014-07-23,True
4,390,223.68001,NaT,2,2016-06-08,False
...,...,...,...,...,...,...
145081,2703430,22918.00600,NaT,8,2016-11-26,False
145082,2703430,0.00000,NaT,2,2014-02-14,False
145083,2703439,219.98401,NaT,0,2016-11-25,True
145084,2703453,44916.64500,2018-05-28,1,2015-05-29,False


pandas 의 `mask` or `np.where` 를 대체할 수 있는 `when` `then` `otherwise` 함수가 존재한다.

In [34]:
%%timeit
df.with_columns(
    pl.when(pl.col('num_group1') == 0)
    .then(pl.lit('VIP'))
    .otherwise(pl.lit('Default'))
    .alias('Customer_category')
)

1.56 ms ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
%%timeit
pd_df['Customer_category'] = pd_df.apply(lambda row: 'VIP' if row['num_group1'] == 0 else 'Default', axis=1)

832 ms ± 5.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Grouping data `group_by`

In [39]:
df.group_by("amount_416A").agg(pl.col('num_group1'))

amount_416A,num_group1
f64,list[i64]
7949.6,[2]
336.752,[0]
906.92804,[0]
605.26404,[0]
10269.586,[0]
202.106,"[0, 0]"
722.598,[0]
539.27,[0]
518.09,"[1, 1, 1]"


여기까지 `polars` 의 기본적인 사용법을 알아보았다. </br>

익숙하고 오래된 pandas 에 비해서 ecosystem 이 적지만, 매우 빠른 속도로 처리되는 것이 놀랍다..!
대회를 진행하며 추가적인 사용법과 `polars` 에 대한 중요한 개념에 대한 학습을 이어나가면 좋을 것 같다.</br>